In [ ]:
library(raster)
library(caret)
library(randomForest)

In [34]:
set.seed(1000)
#loading reference data 
RD.files <- './Data/Reference_Data.shp'
RD <- shapefile(RD.files)

#loading predictors (mean seasonal spectral variations represented by FPCA scores rasters) 

data.folder <- './Data/Predictors'

#file.names <- list.files(data.folder, pattern='*.tif$') #uncomment to take all the predictors
file.names <- list.files(data.folder, pattern='f_') #this takes the NDVI...change the pattern to load the proper predictor
file.paths <- paste(data.folder, file.names, sep="/")

# creating a RasterStack object containing all FPCA scores pixels (all seasonal spectral variations)
predictors <- stack(file.paths)
crs(predictors) <- crs(RD)

In [ ]:
#extract values from predictors at reference data location
pred.val <- extract(predictors,RD, method = 'bilinear')
#creating training data 
dataset <- as.data.frame(pred.val)
dataset$CLASS <- RD$Class

In [ ]:
#downsampling
nc <- length(unique(dataset$CLASS)) #number of target classes
c1 <- length(colnames(dataset)) #number of predictors
sample <- rep(ceiling(0.75*min(table(dataset$CLASS))),nc)

In [ ]:
#Random Forest Recursive Feature Elimination 
subsets <- seq(2,ncol(dataset)-1,3)
set.seed(1000)
ctrl <- rfeControl(functions = rfFuncs,
               method = "repeatedcv",
               number = 10,
               repeats = 5,
               verbose = F)

prProfile <- rfe(x=dataset[, c(1:ncol(dataset)-1)], y=as.factor(dataset$CLASS),
                 sizes = subsets,
                 rfeControl = ctrl)

pr <- predictors(prProfile)[1: prProfile$bestSubset]

In [ ]:
#training the model
set.seed(1000)
ctrl <- trainControl(method = "repeatedcv",number = 10, repeats = 5)
dataset <- dataset[,c(pr,'CLASS')]

#dataset <- dataset
mtry <- 1:sqrt(ncol(dataset))
tunegrid <- expand.grid(.mtry=mtry)
rfDownsampled <- train(as.factor(CLASS) ~ ., data = dataset,
                         method = "rf",
                         ntree = 1500,
                         tuneGrid = tunegrid,
                         metric = "Accuracy",
                         strata = as.factor(dataset$CLASS),
                         trControl = ctrl,
                         sampsize = sample)

In [ ]:
#resampling at 10m
map <- predict(disaggregate(predictors, c(2,2), method = 'bilinear'),rfDownsampled)
#plot map
plot(map)
#export raster
writeRaster(map, './Data/Results/VegetationMap.tif',datatype = 'INT2S')

In [ ]:
#export raster
writeRaster(map, './Data/Results/VegetationMap.tif',datatype = 'INT2S', overwrite=TRUE)
#export results of model
write.table(rfDownsampled$results,dec =',','./Data/Results/TOPORes.txt')
write.table(rfDownsampled$finalModel$confusion,dec =',', './Data/Results/TOPOFinMod.txt')